1.Использовать d a t a s e tM o v i e L e n s
2.Построитьрекомендации(регрессия,предсказываемоценку)нафичах:
    TF-I D Fнатегахижанрах
    Средниеоценки(+m e d i a n ,v a r i a n c e ,etc.)пользователяифильма
3.ОценитьR M S Eнатестовойвыборке

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from math import sqrt


%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

https://analyticsindiamag.com/how-to-build-your-first-recommender-system-using-python-movielens-dataset/

https://github.com/fuxuemingzhu/MovieLens-Recommender

https://blog.codecentric.de/en/2019/07/recommender-system-movie-lens-dataset/

https://github.com/smalec/movielens

https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d

In [3]:
Average_ratings = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
Average_ratings['Count Ratings'] = pd.DataFrame(ratings.groupby('movieId')['rating'].count())
Average_ratings['Min Ratings'] = pd.DataFrame(ratings.groupby('movieId')['rating'].min())
Average_ratings['Max Ratings'] = pd.DataFrame(ratings.groupby('movieId')['rating'].max())
Average_ratings.reset_index(level=0, inplace=True)
Average_ratings.rename(columns={"rating": "Mean Ratings"}, inplace = True)
Average_ratings.head()

,movieId,Mean Ratings,Count Ratings,Min Ratings,Max Ratings
0,1,3.920930,215,0.5,5.0
1,2,3.431818,110,0.5,5.0
2,3,3.259615,52,0.5,5.0
3,4,2.357143,7,1.0,3.0
4,5,3.071429,49,0.5,5.0


In [4]:
Average_tags = pd.DataFrame(tags.groupby('movieId')['tag'].count())
Average_tags.reset_index(level=0, inplace=True)
Average_tags.rename(columns={"tag": "Count tag"}, inplace = True)
Average_tags.head(10)

,movieId,Count tag
0,1,3
1,2,4
2,3,2
3,5,2
4,7,1
5,11,2
6,14,2
7,16,1
8,17,1
9,21,1


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [7]:
#movie_genres = [change_string(g) for g in movies.genres.values]

In [8]:
#movie_genres[:10]

In [9]:
#count_vect = CountVectorizer()
#tfidf_transformer = TfidfTransformer()

In [10]:
#movie_genres_tfidf = tfidf_transformer.fit_transform(
#                                count_vect.fit_transform(movie_genres)
#                    )

In [11]:
#print(movie_genres_tfidf)

In [12]:
movies['genres_str'] = [change_string(g) for g in movies.genres.values]

In [13]:
#movies.head()

In [14]:
vectoriser = TfidfVectorizer()
#movies['genres_Vect'] = list(vectoriser.fit_transform(movies['genres_str']).toarray())

In [15]:
#movies.head()

In [16]:
genres_tfidf_matrix = vectoriser.fit_transform(movies['genres_str'])
genres_tfidf_df = pd.DataFrame(genres_tfidf_matrix.toarray(), index=movies.index.tolist())
print(genres_tfidf_df.shape)

(9742, 20)


In [17]:
#vectoriser.vocabulary_

In [18]:
genres_tfidf_df.columns = vectoriser.get_feature_names()

In [19]:
genres_tfidf_df.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [20]:
movies = movies.merge(genres_tfidf_df, left_index=True, right_index=True)

In [21]:
movies = movies.join(Average_ratings.set_index('movieId'), on = 'movieId')

In [22]:
movies = movies.join(Average_tags.set_index('movieId'), on = 'movieId')

In [23]:
movies.head()

,movieId,title,genres,genres_str,action,adventure,animation,children,comedy,crime,...,romance,scifi,thriller,war,western,Mean Ratings,Count Ratings,Min Ratings,Max Ratings,Count tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy,0.0,0.416846,0.516225,0.504845,0.267586,0.0,...,0.000000,0.0,0.0,0.0,0.0,3.920930,215.0,0.5,5.0,3.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy,0.0,0.512361,0.000000,0.620525,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,3.431818,110.0,0.5,5.0,4.0
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy Romance,0.0,0.000000,0.000000,0.000000,0.570915,0.0,...,0.821009,0.0,0.0,0.0,0.0,3.259615,52.0,0.5,5.0,2.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy Drama Romance,0.0,0.000000,0.000000,0.000000,0.505015,0.0,...,0.726241,0.0,0.0,0.0,0.0,2.357143,7.0,1.0,3.0,NaN
4,5,Father of the Bride Part II (1995),Comedy,Comedy,0.0,0.000000,0.000000,0.000000,1.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,3.071429,49.0,0.5,5.0,2.0


In [24]:
Average_ratings_user = pd.DataFrame(ratings.groupby('userId')['rating'].mean())
Average_ratings_user['Count Ratings U'] = pd.DataFrame(ratings.groupby('userId')['rating'].count())
Average_ratings_user['Min Ratings U'] = pd.DataFrame(ratings.groupby('userId')['rating'].min())
Average_ratings_user['Max Ratings U'] = pd.DataFrame(ratings.groupby('userId')['rating'].max())
Average_ratings_user.reset_index(level=0, inplace=True)
Average_ratings_user.rename(columns={"rating": "Mean Rating U"}, inplace = True)
Average_ratings_user.head()

,userId,Mean Rating U,Count Ratings U,Min Ratings U,Max Ratings U
0,1,4.366379,232,1.0,5.0
1,2,3.948276,29,2.0,5.0
2,3,2.435897,39,0.5,5.0
3,4,3.555556,216,1.0,5.0
4,5,3.636364,44,1.0,5.0


In [25]:
ratings_to_pred = ratings.join(Average_ratings_user.set_index('userId'), on = 'userId')

In [26]:
ratings_to_pred = ratings_to_pred.join(movies.set_index('movieId'), on = 'movieId')
ratings_to_pred.head()

,userId,movieId,rating,timestamp,Mean Rating U,Count Ratings U,Min Ratings U,Max Ratings U,title,genres,...,romance,scifi,thriller,war,western,Mean Ratings,Count Ratings,Min Ratings,Max Ratings,Count tag
0,1,1,4.0,964982703,4.366379,232,1.0,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,...,0.000000,0.0,0.000000,0.0,0.0,3.920930,215.0,0.5,5.0,3.0
1,1,3,4.0,964981247,4.366379,232,1.0,5.0,Grumpier Old Men (1995),Comedy|Romance,...,0.821009,0.0,0.000000,0.0,0.0,3.259615,52.0,0.5,5.0,2.0
2,1,6,4.0,964982224,4.366379,232,1.0,5.0,Heat (1995),Action|Crime|Thriller,...,0.000000,0.0,0.542042,0.0,0.0,3.946078,102.0,1.0,5.0,NaN
3,1,47,5.0,964983815,4.366379,232,1.0,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,...,0.000000,0.0,0.566975,0.0,0.0,3.975369,203.0,0.5,5.0,3.0
4,1,50,5.0,964982931,4.366379,232,1.0,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,...,0.000000,0.0,0.472071,0.0,0.0,4.237745,204.0,1.0,5.0,6.0


In [27]:
ratings_to_pred = ratings_to_pred[['Mean Rating U',
       'action', 'adventure', 'animation', 'children', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'filmnoir', 'horror',
       'imax', 'musical', 'mystery', 'nogenreslisted', 'romance', 'scifi',
       'thriller', 'war', 'western', 'Mean Ratings', 'Count tag', 'rating']]

In [28]:
ratings_to_pred = ratings_to_pred.fillna(0)
ratings_to_pred.rename(columns={"rating": "rating target"}, inplace = True)

In [29]:
ratings_to_pred.head()

,Mean Rating U,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,mystery,nogenreslisted,romance,scifi,thriller,war,western,Mean Ratings,Count tag,rating target
0,4.366379,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,0.0,0.48299,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,3.920930,3.0,4.0
1,4.366379,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.0,0.00000,...,0.000000,0.0,0.821009,0.0,0.000000,0.0,0.0,3.259615,2.0,4.0
2,4.366379,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,0.0,0.00000,...,0.000000,0.0,0.000000,0.0,0.542042,0.0,0.0,3.946078,0.0,4.0
3,4.366379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,...,0.823735,0.0,0.000000,0.0,0.566975,0.0,0.0,3.975369,3.0,5.0
4,4.366379,0.000000,0.000000,0.000000,0.000000,0.000000,0.553854,0.0,0.0,0.00000,...,0.685854,0.0,0.000000,0.0,0.472071,0.0,0.0,4.237745,6.0,5.0


In [30]:
X = ratings_to_pred[['Mean Rating U',
       'action', 'adventure', 'animation', 'children', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'filmnoir', 'horror',
       'imax', 'musical', 'mystery', 'nogenreslisted', 'romance', 'scifi',
       'thriller', 'war', 'western', 'Mean Ratings', 'Count tag']]
Y = ratings_to_pred[['rating target']]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=10)

In [32]:
LG = LinearRegression()
LG.fit(X_train, y_train)
predictions_LG = LG.predict(X_test)
r2_score_LG = r2_score(y_test, predictions_LG) # Оценка качества модели
RMSE_LG = sqrt(mean_squared_error(y_test, predictions_LG))
print(r2_score_LG, RMSE_LG)

0.39947376535432755 0.8102524229446135


In [33]:
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(LG,parameters, cv=None)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

e:\Users\chefranov_as\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.400093955932807
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)


In [34]:
PG = PolynomialFeatures(degree=2)
pol_reg = LinearRegression()

In [35]:
X_train_poly = PG.fit_transform(X_train)
X_test_poly = PG.fit_transform(X_test)
pol_reg.fit(X_train_poly, y_train)
y_poly_pred = pol_reg.predict(X_test_poly)
r2_score_PG = r2_score(y_test,y_poly_pred)
RMSE_PG = sqrt(mean_squared_error(y_test,y_poly_pred))
print(r2_score_PG, RMSE_PG)

0.39797554523755085 0.811262520594625


In [36]:
SVR_ = SVR()

In [37]:
SVR_.fit(X_train, y_train)
predictions_SVR = SVR_.predict(X_test)
r2_score_SVR_ = r2_score(y_test, predictions_SVR) # Оценка качества модели
RMSE_SVR_ = sqrt(mean_squared_error(y_test, predictions_SVR))
print(r2_score_SVR_, RMSE_SVR_)

e:\Users\chefranov_as\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
e:\Users\chefranov_as\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.397448828408965 0.8116173335893967


In [ ]:
GBG = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4,6,8] 
                 }
grid = GridSearchCV(estimator=GBG, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)    

print(grid.best_estimator_)
print(grid.best_score_)
print(grid.best_params_)